# Import Data

In [1]:
import pandas as pd

df = pd.read_csv("/home/nafis/code/research-llm/NLP/finetuning/finetuning-instruct/dummy-data-instruct.csv")
df.head(5)

,prompt,output
0,How do I reset my password in the e-commerce '...,1. Open the app\n2. Go to the login page\n3. T...
1,How can I delete my account in the 'xx' mobile...,1. Open the app\n2. Navigate to 'Settings'\n3....
2,How do I change my phone number in the 'xx' app?,1. Open the app\n2. Go to 'Profile'\n3. Tap on...
3,How can I update my delivery address in the app?,1. Open the app\n2. Navigate to 'Settings'\n3....
4,How do I enable dark mode in the 'xx' mobile app?,1. Open the app\n2. Go to 'Settings'\n3. Tap '...


# Load Model Tokenizer

In [2]:
# 1. Import Dependencies
from unsloth import FastLanguageModel
from transformers import TrainingArguments
import torch
import pandas as pd
import re

# 2. Load Pretrained Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B",  
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    dtype = None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX 4000 SFF Ada Generation. Num GPUs = 3. Max memory: 19.57 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Konversi ke Format Chat Template dan Hitung Panjang Token

In [3]:
# Simpan hasil token count
token_lengths = []

# Simpan hasil format chat
formatted_chats = []

# Loop tiap baris dan proses
for _, row in df.iterrows():
    chat_list = [
        { "role": "user", "content": row["prompt"] },
        { "role": "assistant", "content": row["output"] } 
    ]
    try:
        # Ubah ke format chat string sesuai template Qwen
        formatted = tokenizer.apply_chat_template(chat_list, tokenize=False)
        formatted_chats.append(formatted)

        # Tokenisasi dan hitung panjang token
        tokens = tokenizer(formatted, add_special_tokens=False)["input_ids"]
        token_lengths.append(len(tokens))

    except Exception as e:
        print("Error formatting row:", row)
        print("Exception:", e)

formatted_chats

["<|im_start|>user\nHow do I reset my password in the e-commerce 'xx' mobile app?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1. Open the app\n2. Go to the login page\n3. Tap on 'Forgot Password'\n4. Enter your registered email\n5. Tap 'Submit'\n6. Check your email and reset password<|im_end|>\n",
 "<|im_start|>user\nHow can I delete my account in the 'xx' mobile app?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1. Open the app\n2. Navigate to 'Settings'\n3. Select 'Account'\n4. Tap 'Delete Account'\n5. Enter your PIN to confirm<|im_end|>\n",
 "<|im_start|>user\nHow do I change my phone number in the 'xx' app?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1. Open the app\n2. Go to 'Profile'\n3. Tap on 'Edit Phone Number'\n4. Enter the new number\n5. Verify with OTP<|im_end|>\n",
 "<|im_start|>user\nHow can I update my delivery address in the app?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1. Open the app\n2. Navigate to 'Settings'\n3. T

Format data diatas sudah siap digunakan dalam pipeline training

In [4]:
# Statistik Panjang Token
print("Statistik Panjang Token")
print("----------------------------")
print("Min       :", min(token_lengths))
print("Max       :", max(token_lengths))
print("Rata-rata :", sum(token_lengths) / len(token_lengths))

Statistik Panjang Token
----------------------------
Min       : 48
Max       : 73
Rata-rata : 56.1


Statistik ini akan menjadi acuan dalam menentukan parameter `max_seq_length`.